<a href="https://colab.research.google.com/github/shown5/HandsOnLLM/blob/main/chap9_multimodal_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install matplotlib transformers datasets accelerate sentence-transformers

In [2]:
from urllib.request import urlopen
from PIL import Image

# 雪の中で遊ぶ子犬のAI生成画像を読み込む
puppy_path = "https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/chapter09/images/puppy.png"
image = Image.open(urlopen(puppy_path)).convert("RGB")

caption = "a puppy playing in the snow"

In [3]:
from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel

model_id = "openai/clip-vit-base-patch32"

# テキストを前処理するためのトークナイザーを読み込む
clip_tokenizer = CLIPTokenizerFast.from_pretrained(model_id)

# テキストと画像の埋め込みを生成するためのメインモデルを読み込む
model = CLIPModel.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]